<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Tabular_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#package for the data profiling
%%capture
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [3]:
#package for encoding categorical predictors
%%capture
pip install category_encoders

In [ ]:
from pandas_profiling import ProfileReport

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [5]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving nomad.csv to nomad.csv


In [6]:
# Load Dataframe Dataframe 
df = pd.read_csv('nomad.csv')
df.head(3)

,User ID,User Registered Date,User Num References,User Num Qualified References,User Num Completed PCLs,User Num State Licenses,User Has Compact License (Yes / No),User Count Applications Completed last 7 Days,User Count Applications Completed last 30 Days,User Count Applications Completed,"User Count Jobs Viewed, last 30 Days",User Certifications,User EHR Systems,User Specialization List,User Age Bucket,User Years Nursing Exp,User Years Travel Exp,User First Application Date,User Last Application Date,User Count Offered
0,5f75431f88dc8fa23a15ba1c,2020-09-30,3,2,1,1,Yes,0.0,0.0,1.0,18,ACLS-BLS,Epic-Matrix AOD,"Medical Surgical - Telemetry, Medical-Surgical",35-39,11.0,0.0,2020-10-01,2020-10-01,1
1,5f75395acc3fc01efb9224c7,2020-09-30,3,2,1,1,Yes,0.0,0.0,1.0,1,ACLS,Cerner-Meditech-Soarian,"Medical Surgical - Telemetry, Long Term Care",40s,6.0,0.0,2020-10-07,2020-10-07,0
2,5f751230008c3f36cc830320,2020-09-30,4,3,3,2,Yes,0.0,0.0,6.0,0,ACLS-BLS-NIHSS,Cerner-Epic-HMS-Meditech,"Cardiac ICU, Telemetry, Critical Care, Medical...",35-39,14.0,3.0,2020-09-30,2020-10-14,0


In [ ]:
 #Dataframe Stats
 print(f'df.shape - {df.shape}')
 df.describe()

In [10]:
# Basic Data Quality Check
df.isnull().sum()

User  ID                                            0
User Registered Date                                0
User Num References                                 0
User Num Qualified References                       0
User Num Completed PCLs                             0
User Num State Licenses                             0
User Has Compact License (Yes / No)                 0
User Count Applications Completed last 7 Days     179
User Count Applications Completed last 30 Days    179
User Count Applications Completed                 179
User Count Jobs Viewed, last 30 Days                0
User Certifications                                 9
User EHR Systems                                   22
User Specialization List                          376
User Age Bucket                                     0
User Years Nursing Exp                            115
User Years Travel Exp                             564
User First Application Date                       180
User Last Application Date  

In [12]:
# I made a decion to drop all rows with NaN values since it is unclear how to impute those 
df_cleaned = df.dropna()
df_cleaned.shape

(3761, 20)

In [19]:
df_cleaned.columns #service code to get a list of column, delete before deployment

Index(['User  ID', 'User Registered Date', 'User Num References',
       'User Num Qualified References', 'User Num Completed PCLs',
       'User Num State Licenses', 'User Has Compact License (Yes / No)',
       'User Count Applications Completed last 7 Days',
       'User Count Applications Completed last 30 Days',
       'User Count Applications Completed',
       'User Count Jobs Viewed, last 30 Days', 'User Certifications',
       'User EHR Systems', 'User Specialization List', 'User Age Bucket',
       'User Years Nursing Exp', 'User Years Travel Exp',
       'User First Application Date', 'User Last Application Date',
       'User Count Offered'],
      dtype='object')

In [47]:
# Dataframe of Original Numeric Predictors (dropping all categorical data & target):
X_original = df_cleaned.drop(['User  ID', 'User Registered Date','User Has Compact License (Yes / No)','User Certifications', 
                              'User EHR Systems','User Specialization List','User Age Bucket', 'User First Application Date', 
                              'User Last Application Date','User Count Offered' ], axis=1) 

In [49]:
X_original.shape

(3761, 10)

In [56]:
#Target Dataframe:
Y = df_cleaned['User Count Offered']

In [6]:
#SERVICE CODE TO BE DELETED
df_cleaned = df_cleaned.drop(['User  ID', 'User Registered Date'], axis=1)
df_cleaned.shape 

(3761, 18)

In [33]:
# transforming categorical data
# conversion to dummies for low cardinality variables
df_to_dummies = df_cleaned['User Has Compact License (Yes / No)']
df_dummies = pd.get_dummies(df_to_dummies, columns=['User Has Compact License (Yes / No)'])
# conversion to hashes for high cordinality variables
df_to_hush_transform = df_cleaned[['User Certifications', 'User EHR Systems','User Specialization List']]
hash_encoder = ce.HashingEncoder()
df_hash_encoded = hash_encoder.fit_transform(df_to_hush_transform)

In [44]:
# new variables frame
df_new_variables = pd.concat([df_hash_encoded,df_dummies],axis=1)

In [50]:
# Dataframe of Predictors:
X = pd.concat([X_original,df_new_variables], axis=1)

In [51]:
X.head(3)

,User Num References,User Num Qualified References,User Num Completed PCLs,User Num State Licenses,User Count Applications Completed last 7 Days,User Count Applications Completed last 30 Days,User Count Applications Completed,"User Count Jobs Viewed, last 30 Days",User Years Nursing Exp,User Years Travel Exp,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,No,Yes
0,3,2,1,1,0.0,0.0,1.0,18,11.0,0.0,1,1,0,0,0,0,0,1,0,1
1,3,2,1,1,0.0,0.0,1.0,1,6.0,0.0,0,1,0,0,0,0,2,0,0,1
2,4,3,3,2,0.0,0.0,6.0,0,14.0,3.0,2,1,0,0,0,0,0,0,0,1


In [62]:
# Predictors variables to Numpy following by scaling

#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values, test_size=0.05, random_state=42)

# Using Standard Scaler
scaler = StandardScaler().fit(X.values)
rescaled_X = scaler.transform(X_train)

In [63]:
rescaled_X

array([[ 0.1143412 ,  0.72799446, -0.22934673, ..., -0.76460892,
        -0.42917823,  0.42917823],
       [-0.62838288,  0.72799446, -0.22934673, ..., -0.76460892,
        -0.42917823,  0.42917823],
       [ 0.85706528, -0.07446368, -0.22934673, ...,  0.90342713,
        -0.42917823,  0.42917823],
       ...,
       [-0.62838288, -0.07446368, -0.22934673, ...,  0.90342713,
        -0.42917823,  0.42917823],
       [ 0.1143412 ,  0.72799446,  1.87107466, ..., -0.76460892,
        -0.42917823,  0.42917823],
       [-0.62838288,  0.72799446, -0.22934673, ...,  0.90342713,
        -0.42917823,  0.42917823]])

In [8]:
profile = ProfileReport(df_cleaned, title='Data Profiling Report', html={'style':{'full_width':False}})

In [ ]:
profile.to_notebook_iframe()

# Modelling

In [ ]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

*Linear Regression*

*XGBoost Installation*

In [ ]:
pip install xgboost

In [ ]:
seed = 42
test_size = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [ ]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
